In [377]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Load Hospitalisation Dataset
df = pd.read_csv('../Data/aggregated_cc_by_sa.csv', low_memory=False)
    # Extract only US-state data
df_hosp =  df.loc[df['open_covid_region_code'].str.contains('US-')]
df_hosp = df_hosp.sort_values(by=['date']) 
df_hosp = df_hosp[['open_covid_region_code','region_name','date',
                    'hospitalized_cumulative','hospitalized_new']]
hosp_states = df_hosp['open_covid_region_code'].unique()

# Load Search Trends Dataset
df_st = pd.read_csv('../Data/2020_US_weekly_symptoms_dataset.csv')
st_states = df_st['open_covid_region_code'].unique()

# Clean Up Hospitalisation Dataset
as we iterate through the clean up, we will change dataset variable name by adding _X, where X is version number of the dataset which +1 every time it is handled

#### 1. Remove States with more than 80% of Data = 0.0

In [378]:
count = 1
states = df_hosp['region_name'].unique()
for state in states:
    df_state_hosp = df_hosp.loc[df_hosp['region_name']==state]
    print(count,state, df_state_hosp.shape[0], df_state_hosp['hospitalized_new'].value_counts()[0.0],
         df_state_hosp['hospitalized_cumulative'].value_counts()[0.0])
    count+=1
    if df_state_hosp['hospitalized_new'].value_counts()[0.0] > df_state_hosp.shape[0]*0.8:
        print(state + " has been removed")
        # remove data from state
        df_hosp = df_hosp[(df_hosp.region_name != state)] 

1 Washington 254 126 120
2 Massachusetts 254 61 59
3 New Jersey 235 133 106
4 Virginia 218 31 27
5 Michigan 215 215 215
Michigan has been removed
6 Rhode Island 215 80 43
7 North Carolina 212 212 212
North Carolina has been removed
8 Arizona 212 8 1
9 Illinois 212 212 212
Illinois has been removed
10 South Carolina 212 137 21
11 Wisconsin 212 27 27
12 Georgia 212 22 21
13 Oregon 212 61 18
14 Texas 212 212 212
Texas has been removed
15 Florida 212 19 17
16 California 212 212 212
California has been removed
17 New York 212 137 17
18 New Hampshire 212 74 20
19 District of Columbia 211 211 211
District of Columbia has been removed
20 Nevada 211 211 211
Nevada has been removed
21 Colorado 211 19 16
22 Ohio 211 16 16
23 Tennessee 211 23 20
24 Nebraska 211 106 90
25 Maryland 211 22 21
26 Minnesota 210 18 16
27 Arkansas 210 64 30
28 Pennsylvania 210 198 199
Pennsylvania has been removed
29 West Virginia 210 206 193
West Virginia has been removed
30 Alaska 210 190 180
Alaska has been removed
31

#### 2. Remove Data Points with > XX number of consecutive 0 new hospitalisation days

In [379]:
# Taking Data from Only States with Search Trend Data
states = list(set(df_hosp['region_name'].unique()) & set(df_st['sub_region_1'].unique()))
states

['South Dakota',
 'Montana',
 'Maine',
 'Idaho',
 'New Mexico',
 'Hawaii',
 'Wyoming',
 'North Dakota',
 'New Hampshire',
 'Rhode Island',
 'Nebraska']

In [380]:
# Remove Data Points with > threshold number of consecutive days with 0.0 values
threshold = 14
count = 1
df_hosp_clean = pd.DataFrame()
for state in states:
    print(str(count)+' ' + state)
    df_state = df_hosp.loc[df_hosp['region_name']==state] 
    df_state.insert(5,'hosp_new_iszero',df_state.hospitalized_new == 0.0)
    df_state.insert(6,'group',(df_state.hosp_new_iszero != df_state.hosp_new_iszero.shift()).cumsum())
    df_state.insert(7,'count', df_state.groupby(['hosp_new_iszero', 'group']).cumcount(ascending=False) + 1)
    df_state
    hold = df_state[(df_state['hosp_new_iszero'] == True) & (df_state['count']>threshold)]
    groups_remove = hold['group'].unique()
    count +=1
    for i in groups_remove:
        dates = df_state[(df_state.group == i)]['date']
        dates.min()
        print('Removed Data between:'+ dates.min() +' to '+ dates.max() )
        df_state = df_state[(df_state.group != i)]
    df_hosp_clean= df_hosp_clean.append(df_state)

1 South Dakota
Removed Data between:2020-03-07 to 2020-03-30
2 Montana
Removed Data between:2020-03-07 to 2020-03-25
3 Maine
Removed Data between:2020-03-07 to 2020-03-29
4 Idaho
Removed Data between:2020-03-07 to 2020-03-27
5 New Mexico
Removed Data between:2020-03-06 to 2020-04-13
6 Hawaii
Removed Data between:2020-03-07 to 2020-03-23
7 Wyoming
Removed Data between:2020-03-07 to 2020-03-26
8 North Dakota
9 New Hampshire
Removed Data between:2020-03-04 to 2020-03-23
10 Rhode Island
Removed Data between:2020-03-01 to 2020-04-12
11 Nebraska
Removed Data between:2020-03-05 to 2020-06-02


### Plot Check Cleaned Data

In [403]:
df_plot = df_hosp_clean.loc[df_hosp_clean['region_name']=='North Dakota']
px.line(data_frame=df_plot, x= 'date',y='hospitalized_new',color = 'region_name')

In [382]:
# Extract Region Code For Plotting
df_hosp_clean.loc[:,'open_covid_region_code'] = df_hosp_clean['open_covid_region_code'].str.replace('US-','')

fig = px.choropleth(df_hosp_clean, 
                    locations="open_covid_region_code",
                    color="hospitalized_new", # lifeExp is a column of gapminder
                    hover_name="region_name", # column to add to hover information
                    color_continuous_scale = 'reds',
                    animation_frame="date",
                    range_color=(0, 10),
                    scope = "usa",
                    locationmode="USA-states"
                   )
fig.show()

# Clean Search Trends Dataset
### 1) Remove All Features with < 50% Data

# Merge with Search Trends & Hospitalisation Datasets

In [404]:
states

['South Dakota',
 'Montana',
 'Maine',
 'Idaho',
 'New Mexico',
 'Hawaii',
 'Wyoming',
 'North Dakota',
 'New Hampshire',
 'Rhode Island',
 'Nebraska']

In [405]:
df_st = pd.read_csv('../Data/2020_US_weekly_symptoms_dataset.csv')
df_st=df_st.drop(columns =['country_region_code','country_region',
                           'sub_region_1_code','sub_region_2_code','sub_region_2'])
df_st.insert(3, 'hospitalized_cumulative',np.nan)
df_st.insert(4, 'hospitalized_new',np.nan)

# Drop All States without Hospitalisation Data (16 --> 11 States)
for state in df_st['sub_region_1'].unique():
    if state not in states:
        df_st = df_st[(df_st.sub_region_1 != state)] 

In [406]:
# Extract Hospitalisation Data
for i,j in df_st.iterrows():
    hosp_cumu = df_hosp_clean.loc[(df_hosp_clean['date'] == j['date']) &
                                    (df_hosp_clean['region_name']==j['sub_region_1'])]['hospitalized_cumulative']
    if not hosp_cumu.empty:
        df_st.at[i,'hospitalized_cumulative']= hosp_cumu.values[0]
        
# drop all data points where there is no hospitalisation data
df_st_dropna = df_st.dropna(subset=['hospitalized_cumulative'])

# Make Hospitalized_new column
for state in states:
    df_state = df_st_dropna.loc[df_st_dropna['sub_region_1']==state]
    count = 1
    for i,j in df_state.iterrows():
        if count == 1:
            df_st.at[i,'hospitalized_new']= df_st.loc[i]['hospitalized_cumulative']
            count +=1
        else:
            df_st.at[i,'hospitalized_new'] = df_st.loc[i]['hospitalized_cumulative'] - df_st.loc[i-1]['hospitalized_cumulative']
df_st

,open_covid_region_code,sub_region_1,date,hospitalized_cumulative,hospitalized_new,symptom:Abdominal obesity,symptom:Abdominal pain,symptom:Acne,symptom:Actinic keratosis,symptom:Acute bronchitis,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
114,US-HI,Hawaii,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,US-HI,Hawaii,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,US-HI,Hawaii,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,US-HI,Hawaii,2020-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,US-HI,Hawaii,2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,US-WY,Wyoming,2020-08-24,211.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.87,NaN,NaN
604,US-WY,Wyoming,2020-08-31,215.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.63,NaN,NaN
605,US-WY,Wyoming,2020-09-07,219.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.02,NaN,NaN
606,US-WY,Wyoming,2020-09-14,230.0,11.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.37,NaN,NaN


# Clean Search Trends Dataset
#### 1) Remove All Features with < XX% Data

In [400]:
# remove columns with less than XX% data
df_st_25 = df_st.dropna(thresh=len(df_st)*0.25, axis=1)
df_st_30 = df_st.dropna(thresh=len(df_st)*0.3, axis=1)
df_st_50 = df_st.dropna(thresh=len(df_st)*0.5, axis=1)


# Save Merged Dataset

In [401]:
df_st.to_csv('../Data/USA_coviddata_clean.csv')
df_st_25.to_csv('../Data/USA_coviddata_clean_25.csv')
df_st_30.to_csv('../Data/USA_coviddata_clean_30.csv')
df_st_50.to_csv('../Data/USA_coviddata_clean_50.csv')